In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables from .env file

In [2]:
load_dotenv('../.env/.env')

True

# Retrieve database connection details from environment variables

In [3]:
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')
table_name = 'xdr_data'

# Create the connection string

In [4]:
connection_string = f'postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

# Query the data

In [5]:
query = 'SELECT * FROM xdr_data'
data = pd.read_sql(query, engine)

In [6]:
query = 'SELECT * FROM xdr_data_cleaned'
data_cleaned = pd.read_sql(query, engine)

# Display the data

In [7]:
print(data.head())

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          20247395.0          19111729.0   

In [8]:
print(data_cleaned.head())

      Bearer Id            Start  Start ms              End  End ms  \
0  1.311448e+19   4/4/2019 12:01     770.0  4/25/2019 14:35   662.0   
1  1.311448e+19   4/9/2019 13:04     235.0   4/25/2019 8:15   606.0   
2  1.311448e+19   4/9/2019 17:42       1.0  4/25/2019 11:58   652.0   
3  1.311448e+19   4/10/2019 0:31     486.0   4/25/2019 7:36   171.0   
4  1.311448e+19  4/12/2019 20:10     565.0  4/25/2019 10:40   954.0   

   Dur. (ms)          IMSI  MSISDN/Number          IMEI  \
0  1823652.0  2.082014e+14   3.366496e+10  3.552121e+13   
1  1365104.0  2.082019e+14   3.368185e+10  3.579401e+13   
2  1361762.0  2.082003e+14   3.376063e+10  3.528151e+13   
3  1321509.0  2.082014e+14   3.375034e+10  3.535661e+13   
4  1089009.0  2.082014e+14   3.369980e+10  3.540701e+13   

      Last Location Name  ...  Youtube DL (Bytes)  Youtube UL (Bytes)  \
0  9.16456699548519E+015  ...          15854611.0           2501332.0   
1                L77566A  ...          20247395.0          19111729.0   

In [9]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 149010 non-null  float64
 1   Start                                     150000 non-null  object 
 2   Start ms                                  150000 non-null  float64
 3   End                                       150000 non-null  object 
 4   End ms                                    150000 non-null  float64
 5   Dur. (ms)                                 150000 non-null  float64
 6   IMSI                                      149431 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      149429 non-null  float64
 9   Last Location Name                        148848 non-null  object 
 10  Avg RTT DL (ms)     

In [10]:
print(data_cleaned.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148935 entries, 0 to 148934
Data columns (total 55 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Bearer Id                                 148935 non-null  float64
 1   Start                                     148935 non-null  object 
 2   Start ms                                  148935 non-null  float64
 3   End                                       148935 non-null  object 
 4   End ms                                    148935 non-null  float64
 5   Dur. (ms)                                 148935 non-null  float64
 6   IMSI                                      148935 non-null  float64
 7   MSISDN/Number                             148935 non-null  float64
 8   IMEI                                      148935 non-null  float64
 9   Last Location Name                        148935 non-null  object 
 10  Avg RTT DL (ms)     

# **Task 3** - Experience Analytics

# **Task 3.1**: Aggregate metrics per customer

In [11]:
# Aggregate per customer
aggregated_data = data_cleaned.groupby('IMSI').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Handset Type': lambda x: x.mode().iloc[0] if not x.mode().empty else None
}).reset_index()

# Calculate total TCP retransmission, RTT, and throughput
aggregated_data['Total_TCP'] = (aggregated_data['TCP DL Retrans. Vol (Bytes)'] + aggregated_data['TCP UL Retrans. Vol (Bytes)'])
aggregated_data['Total_RTT'] = (aggregated_data['Avg RTT DL (ms)'] + aggregated_data['Avg RTT UL (ms)'])
aggregated_data['Total_Throughput'] = (aggregated_data['Avg Bearer TP DL (kbps)'] + aggregated_data['Avg Bearer TP UL (kbps)'])

# Drop the intermediary columns
aggregated_data = aggregated_data.drop(columns=[
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)'
])

aggregated_data.head()

,IMSI,Handset Type,Total_TCP,Total_RTT,Total_Throughput
0,2.040471e+14,Quectel Wireless. Quectel Ec21-E,4.267498e+06,63.234638,2.0
1,2.040808e+14,Quectel Wireless. Quectel Ec25-E,4.267498e+06,63.234638,1.0
2,2.082001e+14,Dn Electronics Danew Konnect 350,1.371600e+05,125.795706,109.0
3,2.082001e+14,Samsung Galaxy Grand (Gt-I9060X),2.797900e+04,84.000000,754.0
4,2.082001e+14,Apple iPhone 6S (A1688),4.267498e+06,92.000000,80.5


# **Task 3.2**: Compute and list 10 of the top, bottom, and most frequent:


In [12]:
def compute_top_bottom_frequent(data_cleaned, column, top_n=10):
    top_values = data_cleaned[column].nlargest(top_n).reset_index(drop=True)
    bottom_values = data_cleaned[column].nsmallest(top_n).reset_index(drop=True)
    most_frequent_values = data_cleaned[column].mode()
    
    # If there are fewer than top_n most frequent values, adjust
    if len(most_frequent_values) > top_n:
        most_frequent_values = most_frequent_values.head(top_n)
    
    return {
        'top': top_values,
        'bottom': bottom_values,
        'most_frequent': most_frequent_values
    }

# Compute statistics for each metric
tcp_stats = compute_top_bottom_frequent(aggregated_data, 'Total_TCP')
rtt_stats = compute_top_bottom_frequent(aggregated_data, 'Total_RTT')
throughput_stats = compute_top_bottom_frequent(aggregated_data, 'Total_Throughput')

# Function to print the results in a readable format
def print_stats(stats, title):
    print(f"\n{title}")
    print("\nTop 10 values:")
    print(stats['top'])
    print("\nBottom 10 values:")
    print(stats['bottom'])
    print("\nMost Frequent values:")
    print(stats['most_frequent'])

# Print the results for each metric
print_stats(tcp_stats, "Total TCP Stats")
print_stats(rtt_stats, "Total RTT Stats")
print_stats(throughput_stats, "Total Throughput Stats")


Total TCP Stats

Top 10 values:
0    2.156957e+07
1    2.156957e+07
2    2.156957e+07
3    2.156957e+07
4    2.156957e+07
5    2.156957e+07
6    2.156957e+07
7    2.156957e+07
8    2.156957e+07
9    2.156957e+07
Name: Total_TCP, dtype: float64

Bottom 10 values:
0     97.0
1    128.0
2    129.0
3    134.0
4    143.0
5    176.0
6    176.0
7    177.0
8    179.0
9    182.0
Name: Total_TCP, dtype: float64

Most Frequent values:
0    4.267498e+06
Name: Total_TCP, dtype: float64

Total RTT Stats

Top 10 values:
0    159.0
1    159.0
2    159.0
3    158.0
4    158.0
5    158.0
6    158.0
7    157.0
8    156.0
9    156.0
Name: Total_RTT, dtype: float64

Bottom 10 values:
0     0.0
1     0.0
2     2.0
3     4.0
4     5.0
5     6.0
6     8.0
7     9.0
8     9.0
9    10.0
Name: Total_RTT, dtype: float64

Most Frequent values:
0    63.234638
Name: Total_RTT, dtype: float64

Total Throughput Stats

Top 10 values:
0    51675.0
1    51513.0
2    51440.0
3    51423.0
4    51383.0
5    51230.0
6    51

# **Task 3.3**

# Distribution of the average throughput per handset type

In [13]:
def report_distribution(data_cleaned, group_col, value_col, top_n=10):
    distribution = data_cleaned.groupby(group_col)[value_col].mean()
    top_distribution = distribution.sort_values(ascending=False).head(top_n)
    return top_distribution

throughput_distribution = report_distribution(aggregated_data, 'Handset Type', 'Total_Throughput')

print("\nTop 10 Average Throughput per Handset Type:\n")
print(throughput_distribution)


Top 10 Average Throughput per Handset Type:

Handset Type
Xiaomi Communica. M1803E1A            50942.428647
Xiaomi Communica. Redmi Note 2        49381.000000
Huawei Nova 2I Huawei Mate 10 Lite    47661.000000
Htc 2Q6E100                           47493.000000
Lephone U Pro                         45669.000000
Huawei Stf-Tl10                       45257.000000
Huawei Vns-Dl00                       45143.000000
Xiaomi Communica. B5                  44366.000000
Samsung Galaxy S9 Sm-G960N            44355.000000
Samsung Galaxy On 7                   43773.428647
Name: Total_Throughput, dtype: float64


# Average TCP retransmission view per handset type

In [16]:
def report_distribution(data_cleaned, group_col, value_col, top_n=10):
    distribution = data_cleaned.groupby(group_col)[value_col].mean()
    top_distribution = distribution.sort_values(ascending=False).head(top_n)
    return top_distribution


tcp_distribution = report_distribution(aggregated_data, 'Handset Type', 'Total_TCP')

print("\nTop 10 Average TCP Retransmission per Handset Type:\n")
print(tcp_distribution)


Top 10 Average TCP Retransmission per Handset Type:

Handset Type
Huawei E587 E587U-2 Turkcell Vinnwifi E587    2.156957e+07
Sierra Wireless Usb305                        2.156957e+07
Huawei Gr5                                    2.156957e+07
Huawei Maimang 6                              2.156957e+07
Asustek Asus Zenfone Selfie Zd551Kl           2.156957e+07
Zyxel Communicat. Lte7460                     2.156957e+07
Zyxel Communicat. Sbg3600                     2.156957e+07
Tcl Communicatio. Pixi 4 6 3G Android         2.125147e+07
Samsung Galaxy Core 2 (Sm-G355X)              2.097485e+07
Nubia Technology. Z17 Mini                    2.092174e+07
Name: Total_TCP, dtype: float64


# **Task 3.4** Perform k-means clustering

In [19]:
features = aggregated_data[['Total_TCP', 'Total_RTT', 'Total_Throughput']]

# Apply KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(features)
aggregated_data['cluster'] = kmeans.labels_

numeric_cols = aggregated_data.select_dtypes(include='number').columns


# Describe each cluster with only numeric columns
cluster_description = aggregated_data.groupby('cluster')[numeric_cols].mean()
print("\nCluster Descriptions:")
print(cluster_description)


Cluster Descriptions:
                 IMSI     Total_TCP  Total_RTT  Total_Throughput  cluster
cluster                                                                  
0        2.082016e+14  4.343013e+06  55.562454       3164.015237      0.0
1        2.082016e+14  1.894704e+07  97.383783      21140.289318      1.0
2        2.082016e+14  9.493927e+05  70.881204      14733.470334      2.0
